In [1]:
import os
import glob
import json
import pandas as pd
from pycocotools.coco import COCO
from PIL import Image
import numpy as np
import skimage.io as io
from matplotlib import pyplot as plt
from pprint import pprint
import cv2
#import segmentation_models as sm 
#import tensorflow as tf
from pycocotools import mask as coco_mask 
%matplotlib inline
import math

LOADING AND EXPLORING DATA

In [2]:
with open('dataSet/turtles-data/data/annotations.json', 'r', encoding='utf8') as file:
    annotations = json.load(file)

In [3]:
# initialise COCO API for annotations
coco = COCO('dataSet/turtles-data/data/annotations.json')

loading annotations into memory...
Done (t=15.63s)
creating index...
index created!


In [4]:
# Retrieve all image IDs
img_ids = coco.getImgIds()

In [1]:
data = []
cat_ids = coco.getCatIds()
batch_size = 10

for i in range(0, math.ceil(len(img_ids)), batch_size):
    batch_data = []
    for img_id in range(i, min(i + batch_size, math.ceil(len(img_ids)))):
        img = coco.loadImgs(img_id + 1)[0]
        file_name = "dataSet/turtles-data/data/" + img['file_name']
        image = np.array(Image.open(file_name))

        # Get annotation information
        anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)


        # Create mask for annotations, if any exist
        if anns:
            mask = coco.annToMask(anns[0])
            for i in range(len(anns)):
                mask += coco.annToMask(anns[i])
        else:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        batch_data.append({'input': image, 'expected_mask': mask})
    
    # After processing each batch, add it to the main data list
    data.extend(batch_data)

# Create DataFrame from the collected data
df = pd.DataFrame(data)

NameError: name 'coco' is not defined

In [7]:
df.shape

(4365, 2)

In [8]:
data = []
for i in range(math.ceil(len(img_ids)/2) + 1, len(img_ids), batch_size):
    batch_data = []
    for img_id in range(i, min(i + batch_size, len(img_ids))):
        img = coco.loadImgs(img_id + 1)[0]
        file_name = "dataSet/turtles-data/data/" + img['file_name']
        image = np.array(Image.open(file_name))

        # Get annotation information
        anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)


        # Create mask for annotations, if any exist
        if anns:
            mask = coco.annToMask(anns[0])
            for i in range(len(anns)):
                mask += coco.annToMask(anns[i])
        else:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        batch_data.append({'input': image, 'expected_mask': mask})
    
    # After processing each batch, add it to the main data list
    data.extend(batch_data)


df = df.append(data, ignore_index=True)

: 

: 

In [ ]:
df.shape

In [ ]:
### JUST FOR TESTING IF THE MASK AND INPUT MATCHING IS CORRECT

img = df.iloc[999]['input']          # Get the first image
mask = df.iloc[999]['expected_mask'] # Get the first mask

# Display the image
plt.imshow(img)
plt.title('Image')
plt.axis('off')
plt.show()

# Display the mask
plt.imshow(mask)  # Use alpha for transparency
plt.title('Mask')
plt.axis('off')
plt.show()

IndexError: single positional indexer is out-of-bounds

USING THE PREMADE UNET